In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.conv_learner import *
from fastai.text import *

# from warpctc_pytorch import CTCLoss

/Users/adamschiller/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
PATH = Path('data/IAM_handwriting')
TMP_PATH = PATH/'tmp'

!ls {PATH}

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

# Converting forms to paragraphs

## crop forms via xml info

In [ ]:
import xml.etree.ElementTree as ET
xml = PATH/'xml'

def parse_xml(fname):
    tree = ET.parse(xml/fname)
    root = tree.getroot()
    
    words = []
    for part in root:
        for line in part:
            for word in line.findall('word'):
                for cmp in word:
                    x,y = int(cmp.attrib['x']), int(cmp.attrib['y'])
                    rx,ry = x+int(cmp.attrib['width']), y+int(cmp.attrib['height'])
                    words.append((x,y,rx,ry))
    
    # convert to numpy
    w = np.array(words)
    # create box w/ padding    # need top,left and height,width
    min_x = np.min(w[:,0])-50
    min_y = np.min(w[:,1])-50
    max_x = np.max(w[:,2])+50
    max_y = np.max(w[:,3])+50
    box = (min_x, min_y, max_x, max_y)
    return box

In [ ]:
def crop_and_save(fname, box):
    im = Image.open(PATH/'forms'/fname)
    im = im.crop(box)
    im.save(PATH/'paragraphs'/fname)

In [ ]:
folder = PATH/'forms'
files = []
for file in os.listdir(folder):
    files.append(file)
files.sort()

In [ ]:
os.makedirs(PATH/'paragraphs', exist_ok=True)

In [ ]:
for fname in files:
    box = parse_xml(fname[:-4]+'.xml')
    crop_and_save(fname, box)

In [ ]:
def show_rand_paragraph(fnames):
    r = random.randint(0, len(fnames))
    fname = fnames[r]
    im = Image.open(PATH/'paragraphs'/fname)
    print(fname)
    print(im.size)
    return im

In [ ]:
show_rand_paragraph(files)

# Combining Line labels into paragraph labels

In [ ]:
lines = pd.read_csv(f'{PATH}/ascii/lines.txt', names=['filename','result','value'], escapechar='\\', delim_whitespace=True, skiprows=23, header=None, usecols=[0,1,8])
lines['text'] = lines.apply(lambda row: row.value.replace('|', ' '), axis=1)

In [ ]:
lines.head(30)

In [ ]:
paragraphs = []
for fname in files:
    fname = fname[:-4]
    text = []
    rows = lines[lines.filename.str.startswith(fname+'-')]
    for i,t in rows.text.iteritems():
        text.append(t+' ')
    paragraphs.append((fname+'.png', ''.join(text)))

In [ ]:
len(paragraphs), len(files)

In [ ]:
df = pd.DataFrame(paragraphs, columns=['filename', 'text'])
df.head()

In [ ]:
CSV = PATH/'paragraphs_df.csv'
df.to_csv(CSV, columns=['filename', 'text'], index=False)

# Data Prep

## Chars (~400 chars/paragraph - max: 705)

### Numericalize

In [ ]:
chars = set()
for line in df.text:
    chars = chars.union(set(list(line)))
    
len(chars)

In [ ]:
itos = sorted(list(chars))
itos.insert(0, '_pad_')
itos.insert(1, '_bos_')
itos.insert(2, '_eos_')
itos.insert(3, '_unk_')
len(itos)

In [ ]:
joined_labels = list(df.text)

stoi = collections.defaultdict(lambda: 3, {v:k for k,v in enumerate(itos)})
ids = np.array([np.array([1] + [stoi[letter] for letter in word] + [2]) for word in joined_labels])

In [ ]:
# convert to strings (as labels)
str_ids = np.array([' '.join(str(l) for l in w) for w in ids]).reshape(-1,1)

In [ ]:
df['char_ids'] = str_ids
df.head()

In [ ]:
# length of longest string -> seq_len
df.char_ids.map(lambda x: len(x.split(' '))).mean()
#mean = 386
#max  = 705

In [ ]:
# np.save(TMP_PATH/'paragraph_char_ids.npy', ids)
pickle.dump(itos, open(TMP_PATH/'paragraph_char_itos.pkl', 'wb'))

### CSV

In [6]:
CSV = PATH/'paragraph_chars.csv'
# df.to_csv(CSV, columns=['filename', 'char_ids'], index=False)
csv = pd.read_csv(CSV)

In [7]:
itos = pickle.load(open(TMP_PATH/'paragraph_char_itos.pkl', 'rb'))

In [8]:
val_idxs = np.array(csv.sample(frac=0.15).index)
len(val_idxs)

231

## Word prep  (~120 words/paragraph - max: 170)

### Tokenize/Numericalize

In [ ]:
tokens = Tokenizer().proc_all_mp(partition_by_cores(df.text))

In [ ]:
tokens

In [ ]:
idx = np.argmax([len(o) for o in tokens])
idx, len(tokens[idx]), tokens[idx]

In [ ]:
Image.open(PATH/'paragraphs'/'g07-000a.png')

In [ ]:
np.percentile([len(o) for o in tokens], 98)

In [ ]:
freq = Counter(word for line in tokens for word in line)
freq.most_common(25)

In [ ]:
max_vocab=50000
min_freq=2

In [ ]:
itos = [word for word,count in freq.most_common(max_vocab) if count>min_freq]
itos.insert(0, '_pad_')
itos.insert(1, '_bos_')
itos.insert(2, '_eos_')
itos.insert(3, '_unk_')

stoi = collections.defaultdict(lambda: 3, {v:k for k,v in enumerate(itos)})
ids = np.array([np.array([1]+[stoi[word] for word in line]+[2]) for line in tokens])

In [ ]:
TMP_PATH = PATH/'tmp'
np.save(TMP_PATH/'paragraph_ids.npy', ids)
pickle.dump(itos, open(TMP_PATH/'paragraph_itos.pkl', 'wb'))

In [ ]:
df['ids'] = [' '.join(str(p) for p in o) for o in ids]
df.tail()

In [ ]:
# length of longest string -> seq_len
csv.ids.map(lambda x: len(x.split(' '))).max()

### CSV

In [ ]:
CSV = PATH/'paragraphs.csv'
# df.to_csv(CSV, columns=['filename', 'ids'], index=False)
csv = pd.read_csv(CSV)

In [ ]:
# paragraph_ids = np.load(TMP_PATH/'paragraph_ids.npy')
itos = pickle.load(open(TMP_PATH/'paragraph_itos.pkl', 'rb'))

In [ ]:
val_idxs = np.array(csv.sample(frac=0.15).index)
len(val_idxs)

## Preprocess (rotate, normalize)

In [28]:
f = resnet34
sz = 224
bs = 30

In [29]:
aug_tfms = [RandomRotate(3, mode=1), RandomLighting(0.05, 0.05)]

tfms = tfms_from_model(f, sz, crop_type=CropType.NO, aug_tfms=aug_tfms)
data = ImageClassifierData.from_csv(PATH, 'paragraphs', CSV, bs=bs, val_idxs=val_idxs, tfms=tfms, continuous=True)

In [30]:
# pad ends of lines with pad token for language model
data.aug_dl.pre_pad=False
data.trn_dl.pre_pad=False
data.val_dl.pre_pad=False

### Verify dataset transforms

In [ ]:
denorm = data.trn_ds.denorm
x,y = next(iter(data.aug_dl))
x = denorm(x)

In [ ]:
def label_text(pred):
    ints = to_np(pred).astype(int)
    ints = np.trim_zeros(ints)   # remove padding (0)
    return ' '.join([itos[i] for i in ints])

In [ ]:
def show_img(im, figsize=None, ax=None, alpha=None, title=None):
    if not ax: fig,ax = plt.subplots(figsize=figsize)
    ax.imshow(im, alpha=alpha)
    ax.set_axis_off()
    if title: ax.set_title(title)
    return ax

In [ ]:
fig, axes = plt.subplots(2,1, gridspec_kw={'hspace': 0.5}, figsize=(50,50))
for i,ax in enumerate(axes.flat):
    t = label_text(y[i])
    ax=show_img(x[i], ax=ax, title=t)

## Loss Fn

In [12]:
# categorical cross entropy loss
# list of probabilities for each word in vocab; target is correct word

def seq2seq_loss(input, target):
    target = target.permute(1,0).contiguous()
    tsl = target.size(0)
    sl,bs,nc = input.size()
        
    if sl>tsl: target = F.pad(target, (0,0,0,sl-tsl))
    if tsl>sl: target = target[:sl]
        
    targ = target.view(-1)
    pred = input.view(-1, nc)

    # combination of LogSoftmax and NLLLoss
    return F.cross_entropy(pred, targ.long(), reduction='sum')/bs

## Levenshtein Error Rate

In [13]:
import Levenshtein as Lev

# pulled from Sean Nareen's deepspeech decoder module
# https://github.com/SeanNaren/deepspeech.pytorch/blob/master/decoder.py

def cer(t, p):
    """
    Computes the Character Error Rate, defined as the edit distance.
    Arguments:
        t (string): target space-separated sentence
        p (string): prediction space-separated sentence
    """
    t, p, = t.replace(' ', ''), p.replace(' ', '')
    return Lev.distance(t, p)/len(t)

def wer(s1, s2):
    """
    Computes the Word Error Rate, defined as the edit distance between the
    two provided sentences after tokenizing to words.
    Arguments:
        s1 (string): space-separated sentence
        s2 (string): space-separated sentence
    """

    # build mapping of words to integers
    b = set(s1.split() + s2.split())
    word2char = dict(zip(b, range(len(b))))

    # map the words to a char array (Levenshtein packages only accepts
    # strings)
    w1 = [chr(word2char[w]) for w in s1.split()]
    w2 = [chr(word2char[w]) for w in s2.split()]

    return Lev.distance(''.join(w1), ''.join(w2))

In [14]:
def label_text(pred):
    ints = to_np(pred).astype(int)
    ints = ints[np.nonzero(ints)]
    return ''.join([itos[i] for i in ints])

In [15]:
def char_error_rate(preds, targs):
    bs,sl = targs.size()      #=> ([bs, sl])
    # preds.size()            #=> ([sl, bs, vs])
        
    res = torch.argmax(preds, dim=2)
    error = 0
    for i in range(bs):
        p = label_text(res[:,i])
        t = label_text(targs[i])
        error += cer(t,p)
    return error/bs

# Arch

## Show, Attend, Tell

In [16]:
class Encoder(nn.Module):
    def __init__(self, embed_size, drop=0.4):
        super().__init__()
        
        slices = {128: -4, 256: -3, 512: -2}
        s = slices[embed_size]
        
        net = f(True)
        modules = list(net.children())[:s]    #(8,8,256)
        self.base = nn.Sequential(*modules)

        #mlp
        self.drop = nn.Dropout(drop)
        self.linear1 = nn.Linear(embed_size, embed_size)
        self.linear2 = nn.Linear(embed_size, embed_size)
        
    def forward(self, inp):
        bs,nf,h,w = inp.size()
        
        features = self.base(inp)
        features = features.flatten(2, 3).permute(2, 0, 1)
        
        # initial hidden is an average of features fed through an MLP
        h = self.drop(features)
        h = self.linear2(F.relu(self.linear1(h))).mean(0)
                        
        return features, h

In [17]:
def rand_t(*sz): return torch.randn(sz)/math.sqrt(sz[0])
def rand_p(*sz): return nn.Parameter(rand_t(*sz))

class Attention(nn.Module):
    def __init__(self, embed_size, drop=0.1):
        super().__init__()

        self.attn_1 = nn.Linear(embed_size, embed_size)
        self.attn_2 = nn.Linear(embed_size, embed_size)
        self.drop   = nn.Dropout(drop)
        self.attn_v = rand_p(embed_size)
        
    def forward(self, hidden, feats):
        # this is called once for each timestep
        w1e = self.attn_1(feats)                    # ([64, 30, 256])
        w2h = self.attn_2(hidden)                   # ([30, 256])
        u = torch.tanh(w1e + w2h)                   # ([64, 30, 256])
        u = self.drop(u)
        a = torch.softmax(u @ self.attn_v, 0)       # ([64, 30]) - attention mask
        context = (a.unsqueeze(2) * feats).sum(0)   # ([30, 256]) - weighted sum of features w/ attention
        
        return context, a

In [18]:
class Decoder(nn.Module):
    def __init__(self, embed_size, num_layers, drop=0.3):
        super().__init__()
        
        self.num_layers = num_layers
        self.rnn = nn.GRU(embed_size, embed_size, num_layers)
        if self.num_layers > 1: self.rnn.dropout = drop

    def forward(self, hidden, context):
        _, h = self.rnn(context.unsqueeze(0), hidden.expand(self.num_layers, -1, -1).contiguous())
        
        return h[-1]

In [19]:
class DeepOutputLayer(nn.Module):
    def __init__(self, embed_size, vocab_size, drop=0.3):
        super().__init__()
        
        self.w0 = nn.Linear(embed_size, embed_size)
        self.w1 = nn.Linear(embed_size, embed_size)
        self.w2 = nn.Linear(embed_size, embed_size)
        self.w3 = nn.Linear(embed_size*3, vocab_size)
        self.drop = nn.Dropout(drop)
        
    def forward(self, prev, hidden, context):
        # this is called once for each timestep
        #(30,256)
        a = self.w0(prev)
        b = self.w1(hidden)
        c = self.w2(context)
        out = torch.cat([a,b,c], 1)
        
        return self.w3(self.drop(out))

In [31]:
class Net(nn.Module):
    def __init__(self, hidden_size, vocab_size, seq_len, n_layers=1, drop=0.2):
        super(Net, self).__init__()
        self.seq_len = seq_len
        
        self.encoder = Encoder(hidden_size)
        self.attention = Attention(hidden_size)
        self.decoder = Decoder(hidden_size, n_layers)
        
        self.embed   = nn.Embedding(vocab_size, hidden_size)
        self.drop    = nn.Dropout(drop)
        self.embed.weight.data.uniform_(-0.1, 0.1)

        self.output  = DeepOutputLayer(hidden_size, vocab_size)
        
    def forward(self, x, ret_attn=False):
        bs = x.size(0)
        feats, hidden = self.encoder(x)
        
        res,attns = [],[]
        dec_inp = torch.ones(bs, device=device).long()  #bos token
        
        for i in range(self.seq_len):
            prev = self.drop(self.embed(dec_inp))
            
            context, a = self.attention(hidden, feats)
            hidden = self.decoder(hidden, context)
            out = self.output(prev, hidden, context)
            
#             #bptt
#             if i%30 == 0:
#                 context = repackage_var(context)
#                 a       = repackage_var(a)
#                 hidden  = repackage_var(hidden)
#                 out     = repackage_var(out)
            
            res.append(out)
            attns.append(a)
            dec_inp = out.data.max(1)[1]
            
        res = torch.stack(res)
        if ret_attn: res = res,torch.stack(attns)
        return res

In [32]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))
rnn = Net(256, len(itos), 50, 1)
learn = RNN_Learner(data, SingleModel(to_gpu(rnn)), opt_fn=opt_fn)
learn.clip = 0.25
learn.crit = seq2seq_loss
learn.metrics = [char_error_rate]

# Train

In [33]:
lr=1e-3
learn.fit(lr, 1, cycle_len=1)

# em_sz: 256
# 149.260142 0.978963    seq_len: 50, sz: 500, bs: 10
# 150.115528 0.981975    seq_len: 50, sz: 224, bs: 30

# em_sz: 512, sz: 500, bs: 10


epoch      trn_loss   val_loss   char_error_rate         
    0      161.285217 150.115528 0.981975  



[150.11552810668945, 0.9819754754156605]

In [34]:
learn.fit(lr, 5, cycle_len=1, use_clr=(10, 4))

# em_sz: 256
# 150.466808 0.985582    seq_len: 50, sz: 500, bs: 10
# 149.466808 0.975582    seq_len: 50, sz: 224, bs: 30

epoch      trn_loss   val_loss   char_error_rate         
    0      149.500004 149.762295 0.978805  
    1      149.030719 149.657045 0.978805                
    2      148.803087 149.563997 0.978805                
    3      148.683454 149.429966 0.975582                
    4      148.485525 149.466808 0.975582                



[149.4668083190918, 0.9755815051725074]

# Results

In [24]:
x,y = next(iter(data.val_dl))
learn.model.eval()
preds,attns = learn.model(x, ret_attn=True)   #attns ([sl, 64, bs])
res = torch.argmax(preds, dim=-1)

In [ ]:
seq2seq_loss(preds, y), char_error_rate(preds, y)

In [ ]:
def label_text(pred):
    ints = to_np(pred).astype(int)
    ints = np.trim_zeros(ints)   # remove padding (0)
    return ''.join([itos[i] for i in ints])

In [ ]:
def show_img(im, figsize=None, ax=None, alpha=None, title=None):
    if not ax: fig,ax = plt.subplots(figsize=figsize)
    ax.imshow(im, alpha=alpha)
    ax.set_axis_off()
    if title: ax.set_title(title)
    return ax

In [ ]:
imgs = data.trn_ds.denorm(x)

fig, axes = plt.subplots(2,2, figsize=(10, 10))
for i,ax in enumerate(axes.flat):
#     t = label_text(y[i])     #=>  char indices
    p = label_text(res[:,i])
    ax=show_img(imgs[i], ax=ax, title=p)#t+' '+p)
    
# plt.tight_layout(pad=0.1)

## Attention

In [ ]:
def scale_attn_to_img(att, sz):
    p = 16
    num = sz // p
    a = att.reshape(p,p)
    return np.repeat(np.repeat(a,num, axis=0), num, axis=1)

In [ ]:
idx  = 0
img  = imgs[idx]
pred = res[:,idx]
attn = to_np(attns[:,:,idx])  # first batch

In [ ]:
fig, axes = plt.subplots(6, 3, figsize=(20, 20))
for i,ax in enumerate(axes.flat):
#     ax.plot(attn[i])
    a = scale_attn_to_img(attn[i], 500)
    ax.imshow(img, alpha=None)
    ax.imshow(a, cmap='Blues', interpolation='nearest', alpha=0.3)
    ax.set_title(itos[pred[i].item()])